# Feature selection

See https://git.bia-christian.de/bia/lab_book_VEO/wiki/oxygen

# Correlations between features

See https://git.bia-christian.de/bia/lab_book_VEO/wiki/oxygen#correlation-and-join-features

In [1]:
#Turn off scientific notation
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import numpy as np
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:0.2f}'.format})

#Activate print all activities
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pickle
import zstandard
import time
from datetime import datetime

ts = time.time() 
print("Started script! Loading input file...", datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

#with zstandard.open('/home/bia/Documents/bacterial_phenotypes/connecting_features_abFactors/df_oxygen_kmer9_selected-filterNA.pickle.zst', 
with zstandard.open('/home/bia/Documents/bacterial_phenotypes/connecting_features_abFactors/df_oxygen_gene-families_selected-filterNA.pickle.zst', 
                    'rb') as f:
	df = pickle.load(f)

ts = time.time() 
print("Loaded input pickle zst file...", datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

/tmp/ipykernel_10318/3906458195.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print("Started script! Loading input file...", datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))


Started script! Loading input file... 2024-06-20 14:44:20
Loaded input pickle zst file... 2024-06-20 14:44:21


/tmp/ipykernel_10318/3906458195.py:15: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print("Loaded input pickle zst file...", datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))


In [3]:
df.shape
df.head()

(7255, 20142)

,COG3696@2,COG0606@2,33SZW@2,COG0774@1,COG5009@2,COG0188@2,COG0265@2,COG3270@1,2Z7XP@2,COG1086@2,...,arCOG06650@2157,"Viruses,4R0IQ@35237",arCOG09713@2157,arCOG07735@2157,arCOG07334@2157,arCOG04203@2157,arCOG01811@2157,arCOG00087@2157,arCOG07496@2157,Oxygen tolerance
Best assembly,,,,,,,,,,,,,,,,,,,,,
1002367.3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,anaerobe
108980.91,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,aerobe
1111140.3,1,0,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,aerobe
1112212.3,1,1,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,aerobe
1120962.3,1,1,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,aerobe


In [4]:
# Separar as features (X) e o grupos (y)
#Full dataset:
#X = df.iloc[, :-1]  
#y = df.iloc[:, -1] 

#Subset for developing:
X = df.iloc[0:100, 0:100]
y = df.iloc[0:100, -1] 

In [17]:
X.isnull().any().any()

type(X)

False

pandas.core.frame.DataFrame

In [6]:
# Calcular a correlação de Pearson
correlation_pearson = X.corr()
#correlation_pearson.to_csv('correlation_pearson_total.csv')

In [7]:
correlation_pearson

,COG3696@2,COG0606@2,33SZW@2,COG0774@1,COG5009@2,COG0188@2,COG0265@2,COG3270@1,2Z7XP@2,COG1086@2,...,COG2273@2,32DYM@2,COG0006@2,COG3610@1,COG2985@1,COG2873@2,COG0480@2,COG1109@2,COG2989@2,COG0488@2
COG3696@2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COG0606@2,NaN,1.00,0.07,0.28,0.03,-0.07,0.36,0.29,0.10,0.27,...,0.14,0.07,0.32,0.13,0.29,0.06,-0.07,0.52,0.30,0.24
33SZW@2,NaN,0.07,1.00,0.26,0.03,0.01,0.10,0.21,0.70,0.11,...,0.34,1.00,0.08,0.57,0.21,0.34,0.01,0.06,0.17,0.02
COG0774@1,NaN,0.28,0.26,1.00,0.13,0.04,0.21,0.13,0.37,0.35,...,0.43,0.26,-0.02,0.45,0.21,0.21,0.04,0.24,0.10,0.07
COG5009@2,NaN,0.03,0.03,0.13,1.00,0.30,0.13,0.07,0.05,0.18,...,-0.02,0.03,0.38,0.06,0.07,-0.15,0.30,0.24,0.05,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COG2873@2,NaN,0.06,0.34,0.21,-0.15,0.03,0.07,0.34,0.22,0.02,...,0.46,0.34,-0.01,0.38,0.25,1.00,0.03,0.10,0.15,0.05
COG0480@2,NaN,-0.07,0.01,0.04,0.30,1.00,0.10,0.05,0.01,0.09,...,0.03,0.01,0.13,0.02,0.05,0.03,1.00,0.16,-0.17,-0.02
COG1109@2,NaN,0.52,0.06,0.24,0.24,0.16,0.49,0.12,0.09,0.41,...,0.10,0.06,0.41,0.11,0.06,0.10,0.16,1.00,0.13,0.28
COG2989@2,NaN,0.30,0.17,0.10,0.05,-0.17,0.34,0.59,0.07,-0.08,...,0.15,0.17,0.17,0.16,0.54,0.15,-0.17,0.13,1.00,0.11


In [ ]:
a = [0, 1, 0, 1, 0, 0]
b = [0, 1, 0, 1, 0, 1]

# Create a DataFrame from the two lists
df = pd.DataFrame({'a': a, 'b': b})

# Calculate the correlation matrix
correlation_matrix = df.corr()

In [ ]:
# Calcular a correlação de Spearman
#correlation_spearman = X.corr(method='spearman')


## Join highly correlated features 

https://github.com/ProfHeydeFranca/BacterialData/blob/main/Join_Features.py